# Energy Participation

In [ ]:
####################
##   PARAMETERS   ##
####################

import numpy as np

#Constants used for calculations
e = 1.60218e-19
hbar = 1.05457e-34
phi0 = hbar/(2*e)

#Josephson Energy (Ej) and linear inductance (Lj)

delta_super = (1.8e-4 / 2) * e
#Ej = (0.5 * delta_super * hbar * 2 * np.pi) / ((2*e)**2 * 4444) #value for designed Ej = 8.41 GHz Rn = 8345, Divita test = 4444 Ohm
#Ej_ghz = 15.8251e9
#Ej = 2 * np.pi * hbar * Ej_ghz 
#Lj = phi0**2 / Ej
Lj = 14e-9 
Ej = phi0**2 / Lj

#Enter in Participation Ratio from Simulation
P_qubit = 0.9926 #0.9926
P_cav = 0.0031 #0.002287

#Qiskit_EPR for transmon and resonator P_qubit = 0.9915, P_resonator = 0.002273, qubit_freq = 6.22071e9, res_freq = 9.4382e9, Lj = 10.00e-9
#Qiskit_EPR for transmon and resonator (Strong meshing/higher order) P_qubit = 0.99088, P_resonator = 0.0024975, qubit_freq = 6.45729e9, res_freq = 9.50827e9, Lj = 10.00e-9
#Divita Device Q4 P_qubit = 0.9926, P_resonator = 0.002287, qubit_freq = 6.0714e9, res_freq = 7.5651e9, Lj = 10.345e-9 

#Normalize Participation Ratios
P_qubit_normalised = P_qubit / (P_qubit + P_cav)
P_cav_normalised = P_cav / (P_qubit + P_cav)

#Enter in Frequencies from Simulation
linear_qubit_freq = 6.358e9
linear_res_freq = 9.484e9
omega_qubit = 2 * np.pi * linear_qubit_freq
omega_res = 2 * np.pi * linear_res_freq

####################
##  CALCULATIONS  ##
####################

#phi zero-point fluctuations for qubit
phi_zpf_sq = P_qubit * hbar * 2 * omega_qubit  / (2*Ej)

#anharmonicity qubit (self kerr)
anharm_qubit = P_qubit**2 * hbar * omega_qubit**2 / (8*Ej)

#anharmonicity resonator (self kerr)
anharm_res = P_cav**2 * hbar * omega_res**2 / (8*Ej)

#Charging Energy (Ec)
Ec = anharm_qubit * hbar

#Total Capacitance
C_total = e**2 / (2*Ec)

#cross kerr - dispersive shit (chi)
cross_kerr = (P_qubit * P_cav * hbar * omega_qubit * omega_res) / (4*Ej)

#lamb-shift of qubit frequency
lamb_shift_qubit = anharm_qubit - cross_kerr/2

#lamb-shift of res frequency
lamb_shift_res = anharm_res - cross_kerr/2

#qubit frequency using standard formula (check if calculated frequency makes sense)
qubit_freq_test = (np.sqrt(8*Ej * Ec) - Ec)/ hbar

#detuning, delta
delta = ((linear_res_freq - anharm_res/(2*np.pi)) - (linear_qubit_freq - anharm_qubit/(2*np.pi)))

#resonator-qubit coupling strength (g)
g = np.sqrt(disp_shift_qubit * delta  * (1 + delta/anharm_qubit))

#for comparison with cross kerr (Krantz et al. (2021) p42)
chi_test = (g**2 / delta) * (1 / (1 + delta/(anharm_qubit)))

#test for Ec/Anharmonicity using omega_qubit = sqrt(8 * Ec * Ej)/hbar
Ec_test = (omega_qubit * hbar)**2 / (8*Ej)
anharm_qubit_test = Ec_test/(hbar*2*np.pi*1e6)

#critical photon number
nc = delta**2 / (4*g**2)

#print values
print('--------------')
print('|Calculations|')
print('--------------')
print('Ej:\t\t\t\t\t', '%.4f' % (Ej / (hbar * 2 * np.pi *  1e9)), 'GHz', '\t(' '%.4e' %  (Ej ), 'J)')
print('Ec:\t\t\t\t\t', '%.4f' % (anharm_qubit  / (2*np.pi*1e9)), 'GHz', '\t(' '%.4e' %  (anharm_qubit*hbar), 'J)' )
print('Lj:\t\t\t\t\t', '%.4f' %  (Lj/1e-9), 'nH')
print('Cj:\t\t\t\t\t', '%.4f' %  (C_total/1e-15), 'fF')
print('Participation Ratio Qubit:\t\t', '%.5f' % P_qubit, '\t(Normalized =',  '%.5f' % P_qubit_normalised,')')
print('Participation Ratio Cavity:\t\t', '%.5f' % P_cav, '\t(Normalized =',  '%.5f' % P_cav_normalised,')')
print('Linear Qubit Frequency:\t\t\t', '%.4f' % (linear_qubit_freq/1e9), 'GHz')
print('Linear Res Frequency:\t\t\t', '%.4f' % (linear_res_freq/1e9), 'GHz')
print('Qubit Anharmonicity (alpha_Q):\t\t', '%.4f' % (anharm_qubit / (2*np.pi*1e6)), 'MHz')
print('Res Anharmonicity (alpha_R):\t\t', '%.4f' % (anharm_res / (2*np.pi*1e6)), 'MHz')
print('Dispersive Shift (chi):\t\t\t', '%.4f' % (cross_kerr/ (2*np.pi*1e6)), 'MHz')
print('Qubit Frequency:\t\t\t', '%.4f' % (linear_qubit_freq/1e9 - lamb_shift_qubit/(2*np.pi*1e9)), 'GHz')
print('Res Frequency:\t\t\t\t', '%.4f' % (linear_res_freq/1e9 - lamb_shift_res/(2*np.pi*1e9)), 'GHz')
print('Res-Qubit Coupling (g):\t\t\t', '%.4f' % (g/(2*np.pi*1e6)), 'MHz')
print('Detuning:\t\t\t\t', '%.4f' % (delta/1e9), 'GHz')
print('Ej/Ec:\t\t\t\t\t', '%.4f' % (Ej/Ec))
print('Flux_ZPF_squared:\t\t\t', '%.4f' % (phi_zpf_sq))
print('Critical Photon Number:\t\t\t', '%.4f' % (nc))


In [ ]:
#Constants used for calculations
e = 1.60218e-19
hbar = 1.05457e-34
phi0 = hbar/(2*e)

#get particpation ratios and frequencies
participation_ratios = mode_dict['mat_mode_port']
frequencies = mode_dict['eigenfrequencies']

#get #modes and #junctions from arrays
num_of_modes = np.shape(participation_ratios)[0]
num_of_junctions = np.shape(participation_ratios)[1]

#choose modes user is interested in
modes = np.arange(num_of_modes) + 1 # array of modes
modes_to_compare = np.array([])           # modes to keep
if modes_to_compare.size == 0:
     modes_to_compare = modes
modes_to_delete = np.setdiff1d(modes, modes_to_compare)

#update number of modes left for comparison
num_of_modes = num_of_modes - modes_to_delete.size

#create labels for the modes
dataframe_label_mode = []
for i, mode in enumerate(modes_to_compare):
    dataframe_label_mode.append("mode_" + str(mode))

dataframe_label_junc = []
for i in range(num_of_junctions):
    dataframe_label_junc.append("jj_" + str(i+1))

#delete modes not wanted from participation ratio and frequency vectors
if modes_to_delete.size > 0:
    for i,mode in enumerate(modes_to_delete):
        participation_ratios = np.delete(participation_ratios, (modes_to_delete[i] - (i+1)), axis=0)
        frequencies = np.delete(frequencies, (modes_to_delete[i] - (i+1)), axis=0)

#participation ratio matrix - dimension (num_of_modes, num_of_junctions)
P = np.matrix(np.abs(participation_ratios))

#diagonal matrix for eigenfrequencies
omega_vector = 2* np.pi * frequencies
omega = np.diag(omega_vector)

#Detuning matrix
delta = np.ones((np.shape(omega)[0], np.shape(omega)[0])) #create delta/detuning matrix

#Populate delta/detuning matrix
for j in range(np.shape(omega)[0]):
    delta[:,j] = omega_vector - omega_vector[j]

#Get Ej for each junction
Lj = np.matrix([[13e-9], [9e-9]]) #need function to extract this
Ej = np.diag((phi0**2 / Lj).A1)

#Calcultate Kerr matrix: chi = hbar/4 * (omega*P) * inv_Ej * (omega*P)^T
###Please note all these values are angular frequencies radians/sec, to get frequencies in Hetz divide by 2*pi###
chi = (hbar/4) * (omega*P) * np.linalg.inv(Ej) * np.transpose(omega*P)
diag_matrix = (-1/2) * np.diag(np.ones(num_of_modes)) + np.ones(num_of_modes) #hacky way of creating ones matrix with values of 1/2 down the diagonal
chi_anharm = np.multiply(diag_matrix,chi) #This is the kerr matrix but with the anharmonicity down the diagonal for the self-kerr term
anharm = (1/2) * np.diagonal(chi)
lamb_shift = (1/2) * chi * np.ones((np.shape(chi)[0],1))
renormalised_freqs = np.transpose(np.matrix(frequencies * 2 * np.pi)) - lamb_shift #linear frequencies from eigenmode simulation are renormalised by lamb shift caused by non-linearity

#Get values in mehahertz (MHz) to display to user
chi_freq = chi / (2 * np.pi * 1e6) 
chi_anharm = chi_anharm / (2 * np.pi * 1e6) 
anharm_freq = anharm / (2 * np.pi * 1e6)
lamb_shift_freq = lamb_shift / (2 * np.pi * 1e6)
delta_freq = delta / (2 * np.pi * 1e9)
renormalised_freqs = renormalised_freqs /  (2 * np.pi * 1e9)

#create dataframes and print for user to see
print('Simulation Mode Frequencies:')
freq_df = pd.DataFrame(frequencies / (1e9), dataframe_label_mode, ['Freq (GHz)'])
print(freq_df)
print('______________________________')
print('\n')

print('Renormalised Frequencies:\n ***Freqs from simulation are adjusted for lamb shift')
freq_renorm_df = pd.DataFrame(renormalised_freqs, dataframe_label_mode, ['Freq (GHz)'])
print(freq_renorm_df)
print('______________________________')
print('\n')

print('Participation Ratios:')
ratios_df = pd.DataFrame(np.abs(participation_ratios), dataframe_label_mode, dataframe_label_junc)
print(ratios_df)
print('______________________________')
print('\n')

chi_anharm_df = pd.DataFrame(chi_anharm, dataframe_label_mode, dataframe_label_mode)
print('Chi Matrix (MHz):\n ***Diag is Anharmonicity, Off-Diag is Cross-Kerr')
print(chi_anharm_df)
print('______________________________')
print('\n')

lamb_shift_df = pd.DataFrame(lamb_shift_freq, dataframe_label_mode, ['Lamb Shift (MHz)'])
print('Lamb Shifts:')
print(lamb_shift_df)
print('______________________________')
print('\n')

delta_df = pd.DataFrame(delta_freq, dataframe_label_mode, dataframe_label_mode)
print('Detuning (GHz):')
print(delta_df)
print('______________________________')
print('\n')


In [ ]:
PALACE_Eigenmode_Simulation.calculate_hamiltonian_parameters_EPR(r'C:\PALACE_Simulations\Qiskit_Transmon_Couple_EPR_New\Results', [1,2])